In [1]:
import dolfin as dolf
from math import pi
import numpy as np
from helmholtz_pkg.passive_flame import PassiveFlame
from helmholtz_pkg.flame_transfer_function import n_tau
from helmholtz_pkg.active_flame import ActiveFlame
from helmholtz_pkg.eigensolvers import fixed_point_iteration_pep
from helmholtz_pkg.eigenvectors import normalize_eigenvector
from helmholtz_pkg.mshr import *
import params
import matplotlib.pyplot as plt

In [2]:
def mshr():

    geometry = MeshXDMF('MeshDir/pipe', write_xdmf_file=True)
    geometry()
    mesh = geometry.mesh
    boundaries = geometry.boundaries

    # ________________________________________________________________________________

    def fl_subdomain_func(x):
        x = x[2]
        x_f = params.x_f[0][0]
        a_f = params.a_f
        return x_f - a_f - dolf.DOLFIN_EPS <= x <= x_f + a_f + dolf.DOLFIN_EPS

    subdomains = dolf.MeshFunction('size_t', mesh, mesh.topology().dim())

    subdomains.set_all(0)

    fl_subdomain = dolf.AutoSubDomain(fl_subdomain_func)
    fl_subdomain.mark(subdomains, 1)

   

    return mesh, boundaries, subdomains



In [3]:
degree = 1

mesh, boundaries, subdomains = mshr()

boundary_conditions = {1: {'Robin': params.Y_in},  # inlet
                       2: {'Robin': params.Y_out}}  # outlet



In [4]:
radius = 0.047/2

L = 1
total_volume = np.pi*radius**2*L
print("Total volume of tube is: ",total_volume)

L_flame = params.a_f*2
flame_volume = np.pi*radius**2*L_flame
print("Flame region volume is: ",flame_volume)


def calculate_volume_subdomain(mesh, subdomains):
    V = dolf.FunctionSpace(mesh, 'CG', 2)
    u = dolf.interpolate(dolf.Expression('1',degree = 0), V, )

    dV = dolf.Measure('dx', subdomain_data = subdomains)
    print("Flame Volume: ", dolf.assemble(u*dV(1)))
    
def calculate_whole_domain(mesh):
    volume = dolf.MeshFunction('size_t', mesh, mesh.topology().dim())
    V = dolf.FunctionSpace(mesh, 'CG', 2)
    u = dolf.interpolate(dolf.Expression('1',degree = 0), V, )
    volume.set_all(1)
    dV = dolf.Measure('dx', subdomain_data = volume)
    print("Total Volume: ", dolf.assemble(u*dV(1)))

calculate_volume_subdomain(mesh, subdomains)
calculate_whole_domain(mesh)

Total volume of tube is:  0.0017349445429449633
Flame region volume is:  8.674722714724817e-05
Flame Volume:  3.733834609166547e-05
Total Volume:  0.0017160020582948803



lc = 5e-3, 

| Element size(m) | Flame Region Volume(m3) | Total Volume(m3) | Simulation Time(s)|
| 1e-2 | 7.5096e-05 | 0.0017253 | 776.210 
| 5e-3 | 6.6265e-05 | 0.0016901 | 15.870 
| Stretched | Gaussian | .843 | ---- |